In [1]:
import torch

import torch.nn.functional as F

from loader import get_dataloader
from train import run, run_defense
from models import PointNet, PointCNN, Defense_PointNet

In [2]:
epochs=200
b_size=16
lr=0.001
lr_decay=0.5
lr_decay_step=50
weight_decay=0
eps = 0.1
num_class = 10 #
name = '{}'.format(num_class)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

train_loader, test_loader = get_dataloader(1024, b_size)
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  1
Current cuda device  0
GeForce RTX 2080 Ti


In [3]:
model_a = PointNet(num_class)
PATH = "saves/PointNet.pth"
model_a.load_state_dict(torch.load(PATH))
model_a.eval()

PointNet(
  (conv1): PointConv(local_nn=Sequential(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
  ), global_nn=None)
  (conv2): PointConv(local_nn=Sequential(
    (0): Linear(in_features=67, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
  ), global_nn=None)
  (conv3): PointConv(local_nn=Sequential(
    (0): Linear(in_features=131, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
  ), global_nn=None)
  (lin1): Linear(in_features=256, out_features=256, bias=True)
  (lin2): Linear(in_features=256, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=10, bias=True)
)

In [4]:
#Train
model = Defense_PointNet(num_class)
run_defense(train_loader, test_loader, model, model_a, epochs, b_size, lr,
    lr_decay, lr_decay_step, weight_decay, eps, device)

1/5 cls_loss : 2.1797, discriminator_loss : 0.8210, feature_loss : 0.3239 
2/5 cls_loss : 2.1555, discriminator_loss : 0.8335, feature_loss : 0.3021 
3/5 cls_loss : 2.1606, discriminator_loss : 0.8327, feature_loss : 0.2953 
4/5 cls_loss : 2.1064, discriminator_loss : 0.8229, feature_loss : 0.3004 
5/5 cls_loss : 1.8439, discriminator_loss : 0.8094, feature_loss : 0.2885 
Epoch: 001, Test_cls: 0.1916, Test_D: 0.5000, Best_Test_cls: 0.1916, Duration: 121.09
1/5 cls_loss : 1.7637, discriminator_loss : 0.6904, feature_loss : 0.2954 
2/5 cls_loss : 1.6093, discriminator_loss : 0.5788, feature_loss : 0.3232 
3/5 cls_loss : 1.3984, discriminator_loss : 0.5528, feature_loss : 0.3292 
4/5 cls_loss : 1.2807, discriminator_loss : 0.5401, feature_loss : 0.3334 
5/5 cls_loss : 0.9566, discriminator_loss : 0.5155, feature_loss : 0.3293 
Epoch: 002, Test_cls: 0.5507, Test_D: 0.5380, Best_Test_cls: 0.5507, Duration: 120.39
1/5 cls_loss : 0.9308, discriminator_loss : 0.4918, feature_loss : 0.3377 
2/5

KeyboardInterrupt: 

In [ ]:
#load pretrained
from train import test
PATH = "saves/"+model._get_name() +".pth"
model.load_state_dict(torch.load(PATH))
model.eval()

#test the loaded model
model = model.to(device)
test_acc = test(model,test_loader, device)
print ("test acc: {:.04f}".format(test_acc))